In [49]:
import nibabel as nib
import numpy as np

# Path to an image
path = "/notebooks/model_data_w_resize/dMRI_b0/A10-R01_0028-TT21/DWI_concatenated_b0_resized.nii.gz"
injection_center = "/notebooks/model_data_w_resize/injection_centers/A10-R01_0028-TT21/inj_center.csv"

# Load the image
image = nib.load(path)
data = image.get_fdata()
print(data.shape)

# Load the injection center
injection_center = np.loadtxt(injection_center, delimiter=",")
print(injection_center.shape)

# Turn the data into a torch tensor
injection_center = torch.from_numpy(injection_center).unsqueeze(0).unsqueeze(0).unsqueeze(0).unsqueeze(0).float()
data_torch = torch.from_numpy(data).unsqueeze(0).unsqueeze(0).float()
image_coordinates = torch.from_numpy(np.array([3, 3, 3])).unsqueeze(0).unsqueeze(0).unsqueeze(0).unsqueeze(0).float()
print(injection_center.shape)
print(data_torch.shape)
print(image_coordinates.shape)

(128, 178, 115)
(3,)
torch.Size([1, 1, 1, 1, 3])
torch.Size([1, 1, 128, 178, 115])
torch.Size([1, 1, 1, 1, 3])


In [50]:
from training import *

b0_cube = grab_cube_around_voxel(image=data_torch, voxel_coordinates=[5, 5, 5], kernel_size=16)
b0_cube = torch.from_numpy(b0_cube).float()

print(b0_cube.shape)

torch.Size([1, 1, 32, 32, 32])


In [52]:
class ResnetEncoder(nn.Module):
    
    # Constructor 
    def __init__(self, input_nc, output_nc=1, ngf=64, n_blocks=6, norm_layer=nn.BatchNorm3d, use_dropout=False, 
                 padding_type='reflect', voxel_wise=False):
        """
        Parameters:
            input_nc (int) -- the number of channels in input images
            output_nc (int) -- the number of channels in output images
            ngf (int) -- the number of filters in the last conv layer
            n_blocks (int) -- the number of residual blocks
            norm_layer -- normalization layer
            use_dropout (bool) -- if use dropout layers
            padding_type (str) -- the name of padding layer in conv layers: reflect | replicate | zero
        """

        # Initialize parent class
        super(ResnetEncoder, self).__init__()

        # Initialize the self attributes
        self.input_nc = input_nc
        self.output_nc = output_nc
        self.ngf = ngf
        self.n_blocks = n_blocks
        self.norm_layer = self.get_norm_layer(norm_layer)
        self.use_dropout = use_dropout
        self.padding_type = padding_type
        self.voxel_wise = voxel_wise

        # Whatever this is
        if type(norm_layer) == partial:
            self.use_bias = norm_layer.func == nn.InstanceNorm3d
        else:
            self.use_bias = norm_layer == nn.InstanceNorm3d

        # Define the models
        self.img_model = self.define_img_model()
        self.non_img_model = self.define_non_img_model()
        self.joint_model = self.define_joint_model()

    # Define the model
    def define_img_model(self):
        """
        Define the model architecture
        """

        # Initialize the model and padding size
        model = []
        padding_size = 3
        
        # Define the stride, based on voxel_wise
        if self.voxel_wise:
            stride = 2
        else:
            stride = 1

        # Define the padding layer
        if self.padding_type == 'reflect':
            padding_layer = nn.ReflectionPad3d(padding_size)
        elif self.padding_type == 'replicate':
            padding_layer = nn.ReplicationPad3d(padding_size)
        elif self.padding_type == 'zero':
            padding_layer = nn.ZeroPad3d(padding_size)
        else:
            raise NotImplementedError('padding [%s] is not implemented' % self.padding_type)
        
        # 1. Add the first block
        model.extend([padding_layer, 
                      nn.Conv3d(self.input_nc, self.ngf, kernel_size=7, padding=0, bias=self.use_bias), 
                      self.norm_layer(self.ngf), nn.ReLU(True)])
        
        # 2. Add one convolution
        number_downsampling = 2
        self.number_downsampling = number_downsampling
        mult = 2**number_downsampling
        model += [nn.Conv3d(self.ngf, self.ngf * mult, kernel_size=3,
                    stride=1, padding=1, bias=False),
                        self.norm_layer(self.ngf * mult), nn.ReLU(True)]

        # 3. Add the residual blocks
        for i in range(self.n_blocks):
            model += [ResnetBlock(self.ngf * mult, padding_type=self.padding_type, 
                                  norm_layer=self.norm_layer, use_dropout=self.use_dropout, 
                                  use_bias=self.use_bias)]
            
        # 4. Add more downsampling blocks
        # Cube output: stride 1 | Voxel output: stride 2
        for i in range(number_downsampling):
            mult = 2**(number_downsampling - i)
            model += [nn.Conv3d(self.ngf * mult, int(self.ngf * mult / 2), 
                                kernel_size=3, stride=stride, padding=1, bias=self.use_bias), 
                          self.norm_layer(int(self.ngf * mult / 2)), 
                          nn.ReLU(True)]
            
        # 5. Add another convolutional block for vibes
        # Cube output: stride 1 | Voxel output: stride 2
        model += [nn.Conv3d(int(self.ngf * mult / 2), int(self.ngf * mult / 4),
                            kernel_size=3, stride=stride, padding=1, bias=self.use_bias),
                             self.norm_layer(int(self.ngf * mult / 4)), nn.ReLU(True)]
            
        # 4. Add the last block to make the number of channels as the output_nc and reduce spatial space
        model += [nn.Conv3d(int(self.ngf * mult / 4), self.output_nc, kernel_size=3, stride=2, padding=1, bias=self.use_bias)]
        
        # Cube output: No Adaptive layer | Voxel output: Adaptive layer
        if self.voxel_wise:
            model += [nn.AdaptiveAvgPool3d((1, 1, 1))]
        
        # Return the model
        return nn.Sequential(*model)
    
    # Define the processing for the non-image inputs
    def define_non_img_model(self):
        
        # Stores the model
        model = []
        
        # Add convolutions for the injection centers and image coordinates - expected to have self.output_nc channels
        for i in range(self.number_downsampling):
            model += [nn.Conv3d(self.output_nc, self.output_nc, kernel_size=3, stride=1, padding=1, bias=self.use_bias),
                      self.norm_layer(self.output_nc), 
                          nn.ReLU(True)]
            
        # Return the model
        return nn.Sequential(*model)
            
    # Define joint processing for everything
    def define_joint_model(self):
        
        # Stores the model
        model = []
        
        # Define the factor we multiply by, based on voxel_wise
        if self.voxel_wise:
            factor = 1
        else:
            factor = 3
        
        # Add final convolutions for image and non-image data
        # Cube output: self.output_nc * 3 channels | Voxel output: self.output_nc channels
        for i in range(self.number_downsampling):
            model += [nn.Conv3d(self.output_nc * factor, self.output_nc * factor, kernel_size=3, stride=1, padding=1, 
                                bias=self.use_bias),
                      self.norm_layer(self.output_nc), 
                          nn.ReLU(True)]
            
        # Final convolution to make the number of channels 1
        # Cube output: self.output_nc * 3 channels | Voxel output: self.output_nc channels
        model += [nn.Conv3d(self.output_nc * factor, self.output_nc, kernel_size=3, stride=1, padding=1, bias=self.use_bias)]
        
        # Cube output: No Adaptive layer | Voxel output: Adaptive layer
        if self.voxel_wise:
            model += [nn.AdaptiveAvgPool3d((1, 1, 1))]
            
        # Return the model
        return nn.Sequential(*model)
    
    # Get the normalization layer
    def get_norm_layer(self, norm_layer):

        # If the norm layer is batch norm, we return it
        if "batch" in norm_layer.lower():
            return nn.BatchNorm3d
        elif "instance" in norm_layer.lower():
            return nn.InstanceNorm3d
        else:
            raise NotImplementedError('normalization layer [%s] is not found' % norm_layer)
    
    # Forward pass
    def forward(self, input_x, injection_center, image_coordinates):
        """
        Forward pass
        """
        
        # Define the dimension we concatenate along, depending on voxel wise
        if self.voxel_wise:
            dim = 4
        else:
            dim = 1

        # Do all the convolutions on the cube first
        for layer in self.img_model:
            input_x = layer(input_x)
            print(input_x.shape)
            
        # Do the convolutional layers for the injection center
        injection_center = self.non_img_model(injection_center)
        
        # Do the convolutional layers for the image coordinates
        image_coordinates = self.non_img_model(image_coordinates)
        
        # Concatenate the data along the number of channels
        # Cube output: Dimension 1 | Voxel output: Dimension 4
        input_x = torch.cat((input_x, injection_center), dim=dim)
        input_x = torch.cat((input_x, image_coordinates), dim=dim)
        
        # Do the joint processing
        joint_data = self.joint_model(input_x)
                        
        # Return the model
        return joint_data

In [53]:
model = ResnetEncoder(input_nc=1, ngf=64, n_blocks=3, norm_layer=nn.BatchNorm3d, 
                      padding_type='reflect', voxel_wise=True)
output = model(b0_cube, injection_center, image_coordinates)

print("output shape is: ", output.shape)

torch.Size([1, 1, 38, 38, 38])
torch.Size([1, 64, 32, 32, 32])
torch.Size([1, 64, 32, 32, 32])
torch.Size([1, 64, 32, 32, 32])
torch.Size([1, 256, 32, 32, 32])
torch.Size([1, 256, 32, 32, 32])
torch.Size([1, 256, 32, 32, 32])
torch.Size([1, 256, 32, 32, 32])
torch.Size([1, 256, 32, 32, 32])
torch.Size([1, 256, 32, 32, 32])
torch.Size([1, 128, 16, 16, 16])
torch.Size([1, 128, 16, 16, 16])
torch.Size([1, 128, 16, 16, 16])
torch.Size([1, 64, 8, 8, 8])
torch.Size([1, 64, 8, 8, 8])
torch.Size([1, 64, 8, 8, 8])
torch.Size([1, 32, 4, 4, 4])
torch.Size([1, 32, 4, 4, 4])
torch.Size([1, 32, 4, 4, 4])
torch.Size([1, 1, 2, 2, 2])
torch.Size([1, 1, 1, 1, 1])
output shape is:  torch.Size([1, 1, 1, 1, 1])


In [78]:
# Squeeze the output
output = output.squeeze(0).squeeze(0).detach().numpy()

In [79]:
print(data.shape)
print(output.shape)
print(output)

(256, 356, 230)
(1,)
[-120.82841]


In [2]:
import glob
import os
import nibabel as nib

def glob_files(PATH_NAME, file_format):
    INPUT_FILES = []
    for file in glob.glob(os.path.join(PATH_NAME, os.path.join("**", "*.{}".format(file_format))), recursive=True):
        INPUT_FILES.append(file)
    return INPUT_FILES

nii_gz_files = glob_files("/notebooks/model_data_w_resize", "nii.gz")
b0_images = [file for file in nii_gz_files if "b0" in file and "resized" not in file]
print(len(b0_images))


23


In [42]:
import SimpleITK as sitk

reader = sitk.ImageFileReader()
reader.SetFileName(b0_images[0])
image = reader.Execute()

normalizeFilter = sitk.NormalizeImageFilter()
rescaleFilter = sitk.RescaleIntensityImageFilter()
rescaleFilter.SetOutputMaximum(255)
rescaleFilter.SetOutputMinimum(0)

image = normalizeFilter.Execute(image)
image = rescaleFilter.Execute(image)

array = sitk.GetArrayFromImage(image)
print(array.shape)

image_normalized = sitk.GetImageFromArray(array)

sitk.WriteImage(image_normalized, os.path.join(os.getcwd(), "test2.nii.gz"));

(230, 356, 256)


In [4]:
import nibabel as nib
import numpy as np

image = nib.load(b0_images[0])
data = image.get_fdata()

print(data.shape)

normalized_vector = data / np.linalg.norm(data)

print(normalized_vector.shape)

final_img = nib.Nifti1Image(normalized_vector, image.affine)

nib.save(final_img, os.path.join(os.getcwd(), "nibabel.nii"))

(256, 356, 230)
(256, 356, 230)


In [5]:
path = os.path.join(os.getcwd(), "nibabel.nii")

image = nib.load(path)
data = image.get_fdata()

print(data.shape)

(256, 356, 230)


In [16]:
import numpy as np

path_to_residuals = "/notebooks/tract_residuals/predicted_residuals/epoch_1/image_0.npy"
image0 = np.load(path_to_residuals)
print(image0.shape)

(4, 12, 8, 8, 8, 8)


In [26]:
def to_shape(a, shape):
    y_, x_, z_ = shape
    y, x, z = a.shape
    y_pad = (y_-y)
    x_pad = (x_-x)
    z_pad = (z_-z)
    return np.pad(a,((y_pad//2, y_pad//2 + y_pad%2), 
                     (x_pad//2, x_pad//2 + x_pad%2),
                     (z_pad//2, z_pad//2 + z_pad%2)),
                  mode = 'constant')

# Create random array
random_array = np.random.rand(128, 178, 115)
print(random_array.shape)

(128, 178, 115)


In [27]:
# Define kernel size
kernel_size = 8 * 2

# Get the number of values for each axes that need to be added to fit multiple of kernel
padding_needed = [axis % kernel_size for axis in random_array.shape]
print(padding_needed)

output_shape = []
for i in range(random_array.ndim):
    output_shape.append(random_array.shape[i] + padding_needed[i])
    
print(output_shape)

# Padding the random array to the new shape
random_reshaped = to_shape(random_array, output_shape)
print(random_reshaped.shape)

[0, 2, 3]
[128, 180, 118]
(128, 180, 118)


In [10]:
import nibabel as nib
import numpy as np

path = "/notebooks/model_data_w_resize/dMRI_b0/A10-R01_0028-TT21/DWI_concatenated_b0_resized.nii.gz"
injection_center = "/notebooks/model_data_w_resize/injection_centers/A10-R01_0028-TT21/inj_center.csv"
stream_path = "/notebooks/model_data_w_resize/tckmapped_streamlines/A10-R01_0028-TT21/subtracted_unflipped_resized.nii.gz"

# Load the image
image = nib.load(path)
streamline = nib.load(stream_path)
data = image.get_fdata()
stream_data = streamline.get_fdata()
print(data.shape)
print(stream_data.shape)


(128, 178, 115)
(128, 178, 115)


In [13]:
b0_hemi = data[64:, :, :]
res_hemi = stream_data[64:,:,:]

img = nib.Nifti1Image(b0_hemi, affine=np.eye(4))
img2 = nib.Nifti1Image(res_hemi, affine=np.eye(4))

img_b0 = nib.Nifti1Image(data, affine=np.eye(4))
img_str = nib.Nifti1Image(stream_data, affine=np.eye(4))

nib.save(img, "testingcut.nii")
nib.save(img2, "testingcut_res.nii")
nib.save(img_b0, "ogb0.nii")
nib.save(img_str, "ogres.nii")
